# [Intriguing properties of neural networks](http://arxiv.org/abs/1312.6199)

​This paper reports two counter-intuitive properties:

- It is the **space**, rather than the individual units, that contains the semantic information in the high layers of neural networks.
- We can cause the network to **misclassify an image** by applying **a certain hardly perceptible perturbation**, which is found by maximizing the network’s prediction error. In addition, the specific nature of these perturbations is not a random artifact of learning: the same perturbation can cause a different network, that was trained on a different subset of the dataset, to misclassify the same input.

​We term the so perturbed examples “**adversarial examples**”. These results suggest that the deep neural networks that are learned by backpropagation have nonintuitive characteristics and intrinsic blind spots, **whose structure is connected to the data distribution in a non-obvious way**.

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.utils.data

import torchvision
import torchvision.utils as vutils
import torchvision.transforms as transforms
import torchvision.datasets as dset

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
mnist_train = dset.MNIST('./data', train=True, download=True, transform=transforms.ToTensor())
train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=100, shuffle=True)
mnist_test = dset.MNIST('./data', train=False, download=True, transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=10000, shuffle=False)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class FC(nn.Module):
    def __init__(self):
        super(FC, self).__init__()

        self.layer_1 = nn.Sequential(
            nn.Linear(28*28, 100),
            nn.ReLU()
        )
        
        self.layer_2 = nn.Sequential(
            nn.Linear(100, 100),
            nn.ReLU()
        )
        
        self.layer_3 = nn.Sequential(
            nn.Linear(100, 10)
        )
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        
        out_1 = self.layer_1(x)
        out_2 = self.layer_2(out_1)
        out_3 = self.layer_3(out_2)
    
        return out_3, out_2, out_1

model=FC().to(device)

In [ ]:
loss=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001, weight_decay=0.0001)

In [ ]:
for epoch in range(10):
    for i, (images, labels) in enumerate(train_iter):
        X=images.to(device)
        Y=labels.to(device)

        pre,_,_=model(X)
        cost=loss(pre,Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1)%100==0:
            print('epoch:[%d/%d], iter:[%d/%d], loss:%.5f' % (epoch+1,10,i+1,len(train_iter),cost.item()))
print('Finished Training on the 60000 training images')

In [ ]:
model.eval()

total = 0
correct = 0

for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    
    outputs,_,_ = model(images)
    # torch.max(outputs.data, 1)输出每一行的最大值(value, index)
    _,predicted = torch.max(outputs.data, dim=1)

    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %f %%' % (100 * correct / total))

In [ ]:
def imshow(img):
    image = img.cpu().clone().numpy()
    plt.figure(figsize=(10, 20))
    # image -> (channel, row, col)
    # plt.imshow(row,col,channel)
    plt.imshow(np.transpose(image, (1, 2, 0)))
    plt.axis('off')
    plt.show()

In [ ]:
for i in range(4):
    unit = torch.eye(100)[i,:]

    for images, labels in test_loader:
        images = images.to(device)
        
        _,phi_x,_ = model(images)
        values=torch.mv(phi_x.cpu(),unit)
    
    top_index = np.argsort(values.data.numpy())[-10:]
    top_image= images[top_index]
        
    imshow(torchvision.utils.make_grid(top_image, normalize=True, pad_value=1, nrow=10))

In [ ]:
for i in range(4):
    direction= torch.rand(100)
    
    for images, labels in test_loader:
        images = images.to(device)
        
        _,phi_x,_ = model(images)
        values=torch.mv(phi_x.cpu(),direction)
    
    top_index = np.argsort(values.data.numpy())[-10:]
    top_image= images[top_index]
        
    imshow(torchvision.utils.make_grid(top_image, normalize=True, pad_value=1, nrow=10))

In [ ]:
sample_img=mnist_test[0][0]
output,_,_=model(sample_img.to(device))
_,prediction=torch.max(output.data,1)
imshow(torchvision.utils.make_grid(sample_img, normalize=True))